In [1]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go
px.set_mapbox_access_token(open(".mapbox_token").read())
import plotly.io as pio
pio.renderers.default = "notebook"
from plot_hourly_activity import *

# Minimap

In [2]:
with open('data/locations/raw/Open_Space.geojson') as f:
    data = json.load(f)

open_spaces = pd.read_csv('data/locations/open_spaces.csv')
open_spaces = open_spaces.loc[~open_spaces['Type'].isin(['Cemeteries & Burying Grounds', 'Urban Wilds', 'Open Land', 'Other Open Land'])]

bluebike_stations = pd.read_csv('data/locations/bluebike_stations.csv')
nearest_bluebike_stations = pd.read_csv('nearest_bluebike_stations_k_10_d_0.5.csv')

restaurants = pd.read_csv('data/locations/restaurants.csv')
nearest_restaurants = pd.read_csv('nearest_restaurants_k_10_d_0.5.csv')

food_trucks = pd.read_csv('data/locations/food_trucks.csv')
nearest_food_trucks = pd.read_csv('nearest_food_trucks_k_10_d_0.5.csv')

farmers_markets = pd.read_csv('data/locations/winter_farmers_markets.csv')
nearest_farmers_markets = pd.read_csv('nearest_farmers_markets_k_7_d_0.5.csv')

murals = pd.read_csv('data/locations/murals.csv')
nearest_murals = pd.read_csv('nearest_murals_k_10_d_0.5.csv')

In [3]:
symbols_dict = {
    'Bluebike Stations': 'bicycle',
    'Restaurants': 'restaurant',
    'Food Trucks': 'fast-food',
    'Farmers Markets': 'grocery',
    'Murals': 'art-gallery'
}

In [12]:
def plot_nearest(space_index, nearest_df, peripheral_df, fig, name, verbose=False):
    k = nearest_df.at[space_index, 'num_objs']
    if verbose:
        print(f'{name}: {k}')
    subset_locations = peripheral_df.loc[peripheral_df['ID'].isin(list(nearest_df.loc[space_index, [f'obj{i+1}_id' for i in range(k)]]))]
    fig.add_scattermapbox(lat=subset_locations['Latitude'], lon=subset_locations['Longitude'],
                          hoverinfo='text', hovertext=subset_locations['Name'],
                          marker={'symbol': symbols_dict[name], 'allowoverlap': True, 'size': 15}, name=name)

In [5]:
open_spaces

,SPACEID,Name,Type,Acres,Avg. Lat,Avg. Long
0,1,Wilson Park,"Malls, Squares & Plazas",0.097184,42.340991,-71.145929
1,2,James H. Roberts Playground,Playgrounds,1.004136,42.359526,-71.125332
2,3,Smith Playground,Playgrounds,15.025247,42.365072,-71.131046
3,4,Fern Square,"Malls, Squares & Plazas",0.044509,42.359534,-71.133291
4,5,Ringer Playground,Playgrounds,10.263093,42.350456,-71.138129
...,...,...,...,...,...,...
527,528,Watson Park,Parks,0.106896,42.343725,-71.068584
528,529,Nonquit Street Green,Parks,0.340466,42.318116,-71.068396
529,530,Mahoney Park,Parks,0.381102,42.342008,-71.051382
532,2541,Codman Square,"Malls, Squares & Plazas",0.350710,42.290418,-71.071082


In [14]:
def plot_all_nearest(space_name, show=False, verbose=False):
    space_index = open_spaces.loc[open_spaces['Name'] == space_name].index[0]
    fig = px.choropleth_mapbox(open_spaces, geojson=data['features'][space_index], locations='SPACEID',
                               featureidkey='properties.OBJECTID', hover_name='Name', hover_data={'SPACEID': False},
                               center={'lat': open_spaces.at[space_index, 'Avg. Lat'], 'lon': open_spaces.at[space_index, 'Avg. Long']},
                               zoom=13.4 if open_spaces.at[space_index, 'Acres'] > 15 else 15 if open_spaces.at[space_index, 'Acres'] > 1 else 15.5,
                               width=1080, height=720, opacity=0.4)
    fig.update_traces(marker_line_width=0, showlegend=False)
    plot_nearest(space_index, nearest_bluebike_stations, bluebike_stations, fig, 'Bluebike Stations', verbose=verbose)
    plot_nearest(space_index, nearest_restaurants, restaurants, fig, 'Restaurants', verbose=verbose)
    plot_nearest(space_index, nearest_food_trucks, food_trucks, fig, 'Food Trucks', verbose=verbose)
    plot_nearest(space_index, nearest_farmers_markets, farmers_markets, fig, 'Farmers Markets', verbose=verbose)
    plot_nearest(space_index, nearest_murals, murals, fig, 'Murals')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, legend=go.layout.Legend())
    if show:
        fig.show()
    pio.write_json(fig, f'maps/{space_name.replace("/", "-")}.json')

# IDEA: draw a circle for 5-minute walk time

In [15]:
for i in open_spaces['Name']:
    plot_all_nearest(i)

# Weighted Roundtrip Ratio Bar Graph
Do the thing like Four Seasons

Correlation between weighted roundtrip ratios if you consider trips between two different stations close to an open space as roundtrips or not?

In [7]:
nearest_bluebike_stations['Weighted Roundtrip Ratio'] = nearest_bluebike_stations['Name'].apply(get_weighted_roundtrip_ratio)
nearest_bluebike_stations[['Name', 'Weighted Roundtrip Ratio']]

,Name,Weighted Roundtrip Ratio
0,Wilson Park,0.080705
1,James H. Roberts Playground,0.033004
2,Smith Playground,0.041054
3,Fern Square,0.061990
4,Ringer Playground,0.034217
...,...,...
529,Mahoney Park,0.031290
530,Boundary I,0.000000
531,Sprague Pond,0.000000
532,Codman Square,0.118299


In [8]:
fig = px.bar(nearest_bluebike_stations.sort_values(by='Weighted Roundtrip Ratio'), x='Name', y='Weighted Roundtrip Ratio')
fig.update_layout(
    xaxis_title='Weighted Roundtrip Ratio (proportion of nearby rides that start and end at the same station)',
    yaxis_title='Ratio',
    xaxis={
    'showticklabels': False,
})
fig.show()

# Hourly Activity Plots

In [9]:
for c in np.random.choice(nearest_bluebike_stations['Name'], 25):
    plot_activity(c)

5 nearest stations to Harriet Tubman Square: Columbus Ave at W. Canton St, South End Library - Tremont St at W Newton St, Tremont St at W. Dedham St, Prudential Center - 101 Huntington Ave, Massachusetts Ave at Columbus Ave


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



2 nearest stations to Portsmouth Street Playground: New Balance - 20 Guest St, Boston Landing


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



No stations within 0.5 km of Bellevue Hill Reservation
1 nearest station to Condor Street Beach I: Day Sq


...

1 nearest station to Ronan Park: Bowdoin St at Quincy St


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



2 nearest stations to Central Square: Central Square East Boston, Boston East - 126 Border St


/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/henrywu/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



No stations within 0.5 km of Mirabella Pool
